In [81]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader


import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision import models
from torchmetrics import Accuracy

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
import copy
import gc
import math

import warnings
warnings.filterwarnings('ignore')

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [50]:
epochs = 100

batch_size = 128

classes = ('plane', 'car' , 'bird','cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [52]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
cifar_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,])

In [54]:
train_dataset = torchvision.datasets.CIFAR10(root= './datasets', train = True, download=True, transform = cifar_transforms)
test_dataset =  torchvision.datasets.CIFAR10(root= './datasets', train = False, download=True, transform = cifar_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [55]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True) # num_workers=4,
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

In [58]:
class VGG(nn.Module):
    def __init__(self, features):
        super(VGG,self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512,512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512,512),
            nn.ReLU(True),
            nn.Linear(512,10)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def make_layers(cfg):
    layers = []
    in_channels =3
    for out_channels in cfg:
        if out_channels == 'M':
            layers += [nn.MaxPool2d(kernel_size = 2, stride =2)]
        else:
            conv2d = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding =1)
            layers += [conv2d, nn.ReLU(inplace = True)]
            in_channels = out_channels
    return nn.Sequential(*layers)

cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

def vgg16():
    return VGG(make_layers(cfg))

In [60]:
model = vgg16()
learning_rate = 0.05
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 5e-4)

In [62]:
def evaluate_model(model, data, device):
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = torch.cat([logits, model(x_batch)])
            targets = torch.cat([targets, y_batch])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [64]:
def train_normal(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [66]:
model_normal = train_normal(model = model.to(device), epochs = epochs)

Epoch 1/100 - test accuracy: 23.50% and CE loss 1.84
Epoch 2/100 - test accuracy: 32.76% and CE loss 1.60
Epoch 3/100 - test accuracy: 44.23% and CE loss 1.64
Epoch 4/100 - test accuracy: 58.82% and CE loss 1.02
Epoch 5/100 - test accuracy: 60.55% and CE loss 0.93
Epoch 6/100 - test accuracy: 63.02% and CE loss 1.11
Epoch 7/100 - test accuracy: 69.27% and CE loss 1.06
Epoch 8/100 - test accuracy: 74.10% and CE loss 0.71
Epoch 9/100 - test accuracy: 74.67% and CE loss 0.67
Epoch 10/100 - test accuracy: 75.15% and CE loss 0.97
Epoch 11/100 - test accuracy: 78.07% and CE loss 0.62
Epoch 12/100 - test accuracy: 79.47% and CE loss 0.62
Epoch 13/100 - test accuracy: 77.30% and CE loss 0.50
Epoch 14/100 - test accuracy: 78.25% and CE loss 0.50
Epoch 15/100 - test accuracy: 77.78% and CE loss 0.66
Epoch 16/100 - test accuracy: 80.02% and CE loss 0.55
Epoch 17/100 - test accuracy: 80.84% and CE loss 0.42
Epoch 18/100 - test accuracy: 80.25% and CE loss 0.37
Epoch 19/100 - test accuracy: 81.33% 

In [68]:
# Model to GPU and eval mode.
model_normal.to(device)
model_normal.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 85.55%


In [70]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "vgg_cifar.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_normal.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\vgg_cifar.pth


# Adversarial Training

In [73]:
model = vgg16()
learning_rate = 0.05
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 5e-4)
eps= [0.01, 0.03, 0.06, 0.1, 0.3, 0.5]

In [75]:
def evaluate_model(model, data, device):
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = torch.cat([logits, model(x_batch)])
            targets = torch.cat([targets, y_batch])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [77]:
def train_adv(model, epsilon, epochs):
    model.train()
    eps = epsilon
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            x_batch = projected_gradient_descent(model, x_batch, eps, eps/10, 40, np.inf)
            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [83]:
model_adversarial = train_adv(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/100 - test accuracy: 21.61% and CE loss 1.97
Epoch 2/100 - test accuracy: 33.43% and CE loss 1.86
Epoch 3/100 - test accuracy: 34.10% and CE loss 1.79
Epoch 4/100 - test accuracy: 53.47% and CE loss 1.36
Epoch 5/100 - test accuracy: 61.15% and CE loss 1.08
Epoch 6/100 - test accuracy: 67.06% and CE loss 1.02
Epoch 7/100 - test accuracy: 69.84% and CE loss 0.79
Epoch 8/100 - test accuracy: 70.26% and CE loss 0.79
Epoch 9/100 - test accuracy: 74.50% and CE loss 0.68
Epoch 10/100 - test accuracy: 76.50% and CE loss 0.62
Epoch 11/100 - test accuracy: 77.97% and CE loss 0.81
Epoch 12/100 - test accuracy: 77.82% and CE loss 0.66
Epoch 13/100 - test accuracy: 75.05% and CE loss 0.64
Epoch 14/100 - test accuracy: 75.97% and CE loss 0.68
Epoch 15/100 - test accuracy: 79.45% and CE loss 0.62
Epoch 16/100 - test accuracy: 77.86% and CE loss 0.55
Epoch 17/100 - test accuracy: 80.09% and CE loss 0.72
Epoch 18/100 - test accuracy: 78.69% and CE loss 0.75
Epoch 19/100 - test accuracy: 82.92% 

In [85]:
# Model to GPU and eval mode.
model_adversarial.to(device)
model_adversarial.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adversarial, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 86.29%


In [87]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "vgg_cifar_adv.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_adversarial.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\vgg_cifar_adv.pth
